In [1]:
import csv
import pandas as pd
from collections import Counter

In [2]:
raw_data = pd.read_csv('data/raw_data.csv',encoding='ISO-8859-1')

In [3]:
data = raw_data.copy()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null object
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [4]:
data.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,12/9/2011 12:50,4.95,12680.0,France


### 缺失值处理

In [5]:
data.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

发现'产品描述'和'客户编号'存在缺失值,产品描述缺失1454条数据，缺失率0.27%,这里我们主要研究客户行为,产品描述缺失不影响我们后续分析,所以暂不处理；而客户编号缺失135080条,在实际业务中客户ID缺失率高达24.9%,由于我们无法知道缺失原因,所以暂时把缺失值替换成0

In [6]:
data['CustomerID'].fillna(0,inplace=True)
data['CustomerID'].isnull().sum()

0

### 删除重复值

In [7]:
data.drop_duplicates(keep='first',inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 536641 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      536641 non-null object
StockCode      536641 non-null object
Description    535187 non-null object
Quantity       536641 non-null int64
InvoiceDate    536641 non-null object
UnitPrice      536641 non-null float64
CustomerID     536641 non-null float64
Country        536641 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 36.8+ MB


共删除5226条重复数据

### 异常值处理

In [8]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,536641.000000,536641.000000,536641.000000
mean,9.620029,4.632656,11435.904653
std,219.130156,97.233118,6795.044250
min,-80995.000000,-11062.060000,0.000000
25%,1.000000,1.250000,0.000000
50%,3.000000,2.080000,14336.000000
75%,10.000000,4.130000,16241.000000
max,80995.000000,38970.000000,18287.000000


In [9]:
data[data['Quantity']<= 0].head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,12/1/2010 9:41,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,12/1/2010 9:49,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,12/1/2010 10:24,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom


In [10]:
data[data['UnitPrice']<=0].head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,12/1/2010 11:52,0.0,0.0,United Kingdom
1970,536545,21134,NaN,1,12/1/2010 14:32,0.0,0.0,United Kingdom
1971,536546,22145,NaN,1,12/1/2010 14:33,0.0,0.0,United Kingdom
1972,536547,37509,NaN,1,12/1/2010 14:33,0.0,0.0,United Kingdom
1987,536549,85226A,NaN,1,12/1/2010 14:34,0.0,0.0,United Kingdom


我们看到单价和数量存在有负值和零值的情况,我们预计为退货/促销商品,由于不在研究目的的范围内,我对这些数据采取了删除操作

In [11]:
data = data[(data['UnitPrice']>0) & (data['Quantity']>0)]

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 524878 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      524878 non-null object
StockCode      524878 non-null object
Description    524878 non-null object
Quantity       524878 non-null int64
InvoiceDate    524878 non-null object
UnitPrice      524878 non-null float64
CustomerID     524878 non-null float64
Country        524878 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 36.0+ MB


经过异常值处理后还剩524878条有效数据

### 增加字段

数据集中只有单价和数量,缺少总消费金额,所以我们添加一列

In [13]:
data['Amount'] = data.apply(lambda x: x.UnitPrice*x.Quantity, axis=1)

In [14]:
data.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Amount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


### 更改日期时间的数据格式

In [32]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 524878 entries, 0 to 524877
Data columns (total 9 columns):
InvoiceNo      524878 non-null object
StockCode      524878 non-null object
Description    524878 non-null object
Quantity       524878 non-null int64
InvoiceDate    524878 non-null datetime64[ns]
UnitPrice      524878 non-null float64
CustomerID     524878 non-null float64
Country        524878 non-null object
Amount         524878 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 36.0+ MB


In [25]:
data.to_csv('data/data.csv',index=None)